In [0]:
#display(dbutils.fs.ls('dbfs:/FileStore/tables'))
display(dbutils.fs.ls('/databricks-datasets/songs'))

path,name,size,modificationTime
dbfs:/databricks-datasets/songs/README.md,README.md,1719,1455505885000
dbfs:/databricks-datasets/songs/data-001/,data-001/,0,1699763348429
dbfs:/databricks-datasets/songs/data-002/,data-002/,0,1699763348429


In [0]:
f = open('/dbfs/databricks-datasets/songs/README.md', 'r')
print(f.read())

Sample of Million Song Dataset

## Source
This data is a small subset of the [Million Song Dataset](http://labrosa.ee.columbia.edu/millionsong/).
The original data was contributed by The Echo Nest.
Prepared by T. Bertin-Mahieux <tb2332 '@' columbia.edu>

## Attribute Information
- artist_id:string
- artist_latitude:double
- artist_longitude:double
- artist_location:string
- artist_name:string
- duration:double
- end_of_fade_in:double
- key:int
- key_confidence:double
- loudness:double
- release:string
- song_hotnes:double
- song_id:string
- start_of_fade_out:double
- tempo:double
- time_signature:double
- time_signature_confidence:double
- title:string
- year:double
- partial_sequence:int

## Citation
Using the dataset?

Please cite the following paper [pdf](http://www.columbia.edu/~tb2332/Papers/ismir11.pdf) [bib](http://labrosa.ee.columbia.edu/millionsong/sites/default/files/millionsong_ismir11_1.bib):

Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere. 
The M

In [0]:
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructType, StructField

# Define variables used in the code below
file_path = "/databricks-datasets/songs/data-001/"
table_name = "raw_song_data" # Replace "your_table_name" with the desired table name
checkpoint_path = "/tmp/pipeline_get_started/_checkpoint/song_data"

schema = StructType(
  [
    StructField("artist_id", StringType(), True),
    StructField("artist_lat", DoubleType(), True),
    StructField("artist_long", DoubleType(), True),
    StructField("artist_location", StringType(), True),
    StructField("artist_name", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("end_of_fade_in", DoubleType(), True),
    StructField("key", IntegerType(), True),
    StructField("key_confidence", DoubleType(), True),
    StructField("loudness", DoubleType(), True),
    StructField("release", StringType(), True),
    StructField("song_hotnes", DoubleType(), True),
    StructField("song_id", StringType(), True),
    StructField("start_of_fade_out", DoubleType(), True),
    StructField("tempo", DoubleType(), True),
    StructField("time_signature", DoubleType(), True),
    StructField("time_signature_confidence", DoubleType(), True),
    StructField("title", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("partial_sequence", IntegerType(), True)
  ]
)

(spark.readStream
  .format("cloudFiles")
  .schema(schema)
  .option("cloudFiles.format", "csv")
  .option("sep","\t")
  .load(file_path)
  .writeStream
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True)
  .toTable(table_name)
)